In [ ]:
import time


In [ ]:
for i in range(0,5):
    print(i+1, ": Praying to code almighty for a good grade." + "I hope I don't get a meh score.")
    time.sleep(0.6)
print("\n \nHope github doesn't crash.")
print("Hope Huggingface doesn't crash.")
print("Hope my computer doesn't crash.")
print("Hope I don't crash.")
print("Hope all the above prayers work.")

In [ ]:
#install pytorch if you dont have it(cpu/cuda)
#%pip install torch torchvision
#%pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [1]:
%pip install transformers
%pip install scipy
%pip install gradio
%pip install gradio plotly yfinance


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: scipy in c:\users\sudhanvasp\appdata\local\programs\python\python311\lib\site-packages (1.10.1)




[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/65.3 kB ? eta -:--:--
     ---------------------------------------- 65.3/65.3 kB 3.7 MB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
                                              0.0/3.8 MB ? eta -:--:--
     ---                                      0.4/3.8 MB 11.2 MB/s eta 0:00:01
     --------------                           1.4/3.8 MB 17.2 MB/s eta 0:00:01
     ---------------------------------------  3.8/3.8 MB 30.1 MB/s eta 0:00:01
     ---------------------------------------- 3.8/3.8 MB 26.7 MB/s eta 0:00:00
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.28.2
    Uninstalling requests-2.28.2:
      Successfully uninstalled requests-2.28.2
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import gradio as gr
import yfinance as yf
import plotly.express as px
from plotly.offline import plot


In [12]:
def get_stock_data(stock_symbol, period="1d"):
    stock = yf.Ticker(stock_symbol)
    data = stock.history(period=period)
    return data

# Function to visualize stock data using Plotly
def visualize_stock_data(stock_data):
    fig = px.line(stock_data, x=stock_data.index, y="Close", title="Stock Price Trend")
    plot_div = plot(fig, output_type="div", include_plotlyjs=False)
    return plot_div

The model was trained when username was denoted as @user and the links are replaced with http. So we bow down to the arbitary model throwing tantrums and satisfy its wishes

In [7]:

# preprocess tweet as function
def preprocess(tweet):
    tweet_words = []

    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)
    return tweet_proc


In [8]:
# load model and tokenizer
#roberta = "cardiffnlp/twitter-roberta-base-sentiment" this is the base model
roberta = "sudhanvasp/Sentiment-Analysis" #this is the custom model that utilizes the base model with additional parameters to improve accuracy and performance


model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']


Some weights of the model checkpoint at sudhanvasp/Sentiment-Analysis were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#output as a function if you dont want gradio
def sentiment(encoded_tweet):
        tweet_proc = preprocess(tweet)
        encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')

        output = model(**encoded_tweet)

        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        #storing labels and scores in a dictionary in one line
        results = dict(zip(labels, scores))
         #print max score and label of the tweet
        print("Tweet: ", tweet)
        print("Sentiment: ", labels[scores.argmax()], scores.max())
        print("\n\n")
        print("In detail: ")
        return results
def nograd():
        for i in range(0,5):
                tweet = input("Enter a tweet: ")
                tweet_proc = preprocess(tweet)
                encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
                sentiment(encoded_tweet)
                        

        
    

In [9]:


# Define the Gradio interface
def predict_sentiment(tweet):
    tweet_proc = preprocess(tweet)
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')

    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    # Get the sentiment label and score
    sentiment_label = labels[scores.argmax()]
    sentiment_score = scores.max()

    return sentiment_label, sentiment_score


In [ ]:

# Create a Gradio interface
iface = gr.Interface(
    fn=predict_sentiment,
    inputs="text",
    outputs=[gr.components.Textbox(label="Sentiment Label"),gr.components.Textbox(label="Sentiment Score")],
    title="Sentiment Analysis",
    description="Enter a tweet, and this model will predict its sentiment.",
)

# Launch the Gradio interface
#iface.launch(share=True)
iface.launch()

In [18]:
sentiment_iface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.inputs.Textbox(label="Enter a tweet for Sentiment Analysis"),
    outputs=[gr.outputs.Textbox(label="Sentiment Label"), gr.outputs.Textbox(label="Sentiment Score")],
    live=True,
    title="Sentiment Analysis",
    description="Enter a tweet to predict sentiment.",
)

# Create Gradio interface for stock data visualization
stock_iface = gr.Interface(
    fn=get_stock_data,
    inputs=gr.inputs.Textbox(label="Enter Stock Symbol (e.g., AAPL) for Stock Trends"),
    outputs=gr.outputs.HTML(label="Stock Price Trend"),
    live=True,
    title="Stock Market Trends",
    description="Enter a stock symbol to visualize its price trend.",
)

# Launch the Gradio interfaces
sentiment_iface.launch()
stock_iface.launch()

C:\Users\sudhanvasp\AppData\Local\Temp\ipykernel_30752\1094688662.py:3: GradioDeprecationWarning:

Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components

C:\Users\sudhanvasp\AppData\Local\Temp\ipykernel_30752\1094688662.py:3: GradioDeprecationWarning:

`optional` parameter is deprecated, and it has no effect

C:\Users\sudhanvasp\AppData\Local\Temp\ipykernel_30752\1094688662.py:3: GradioDeprecationWarning:

`numeric` parameter is deprecated, and it has no effect

C:\Users\sudhanvasp\AppData\Local\Temp\ipykernel_30752\1094688662.py:4: GradioDeprecationWarning:

Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components

C:\Users\sudhanvasp\AppData\Local\Temp\ipykernel_30752\1094688662.py:13: GradioDeprecationWarning:

Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.com

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
